# 01.01 EDA y Modelo Inicial
Este programa muestra como cargar un archivo, hacer un mínimo tratamiento y exploración de datos. Plantea algunas reglas de clasificación básica y evalúa los resultados.

In [ ]:
#Se limpia la memoria
rm(list=ls())
gc()

In [ ]:
#La librería data table permite manejar grandes volumenes de datos en memoria
library("data.table")

#Contiene funciones para evaluar performance de la clasificación
library("ROCR")

#Permite generar graficos y visualizaciones
library(ggplot2)

In [ ]:
setwd("/Users/rosariotoraglio/Desktop/MaestriaCienciadeDatos/Laboratorio1/clase1/")


#Parametros entrada

#kcarpeta_datasets    <- "../input/laboratorio-de-implementacion-i-2021/"   #KAGGLE
kcarpeta_datasets    <- "./datasets/"                          #VM o Ubuntu

#Archivo con datos etiquetados para entrenamiento
karchivo_entrada      <-  paste0(kcarpeta_datasets, "competencia1_2022.csv")

#Separador de campos en archivos
kcampos_separador     <-  "\t"

#Campo que identifica las muestras
kcampo_id             <-  "numero_de_cliente"

#Campo que contiene la clase a estimar
kclase_nomcampo       <-  "clase_ternaria"

#Valor de interés
kclase_valor_positivo <-  "BAJA+2"

#Campos a borrar para el entrenamiento
kcampos_a_borrar      <-  c(kcampo_id,kclase_nomcampo,"foto_mes")

#Campo que contendrá a la variable objetivo generada
kobjetivo             <-  "clase"

#Identificación del modelo
kmodelo               <-  "01-EDA"

#Ganancia por TP
kTPGain               <-  78000

#Pérdida por FP
kFPGain               <-  -2000


## Lectura y manejo básico de datos

In [ ]:
#Lectura de archivo de en un data frame
dataset <- fread(karchivo_entrada)

### Estadísticas básicas del dataset

In [ ]:
summary(dataset[,1:10]) #Primeras 10 columnas

In [ ]:
#El universo agrupado por  clase_ternaria
ftable(dataset$clase_ternaria)

In [ ]:
ftable(dataset[,c('foto_mes','clase_ternaria')])

In [ ]:
dtrain = dataset[ foto_mes==202101]
dapply = dataset[ foto_mes==202103]



# Selección de datos
Los data frames pueden filtrarse especificando entre corchetes las filas y columnas [filas,columnas]. Las formas de especificar mas usadas son:
- Índice numérico, por ejemplo, *1:10* indica del elemento 1 al 10 (inclusive)
- Nombre o lista de nombres, por ejemplo, *campo*
- Lista de valores Verdadero/Falso. Es utilizadon con condiciones del tipo *columna == valor*


In [ ]:
#Seleccionar 3 primeras filas y campos indicados. La función c() genera una lista de valores
dtrain[1:3,c("Master_mconsumototal","clase_ternaria")]

In [ ]:
dtrain[1:5,c(1:5,155)]

In [ ]:
#Seleccion por un campo indirecto. kclase_nomcampo es una variable que tiene el nombre del campo. 
dtrain[1:3,..kclase_nomcampo]

Para seleccionar en base a una condicion se arma una lista con Verdadero/Falso actuando como flitro de línea. La siguiente lista adopata el valor Verdadero para las líneas BAJA+2

In [ ]:
c(dtrain[,..kclase_nomcampo] == kclase_valor_positivo)

Pasando el filtro como condición de línea al data frame se pueden seleccionar solo las que cumplen con la condición

In [ ]:
dtrain[c(dtrain[,..kclase_nomcampo] == kclase_valor_positivo),][1:10]

### Modificación de datos

Puedo asignar valores a una columna del data frame con el operador := En este caso la referencia indirecta debe hacerse entre paréntesis

In [ ]:
#Creo una columna con valor constante
dtrain[,(kobjetivo) := 0]
#dtrain[  , clase:= 0 ]   Es lo mismo! kobjetivo se definio igual al string "clase"

Asigno 1 en caso de BAJA+2 y 0 para cualquier otro valor.

In [ ]:
#clase
dtrain[ clase_ternaria=='BAJA+2' ,   clase:= 1]
sum( dtrain$clase )

Puedo eliminar columnas asignandoles el valor NULL

In [ ]:
#Borro una o varias columnas
#dtrain[ ,  (kcampos_a_borrar) := NULL    ] 

In [ ]:
#calculo la ganancia de incentivar cada caso
dtrain[  , ganancia:= kFPGain ] #-1250
dtrain[ clase_ternaria=='BAJA+2' ,   ganancia:= kTPGain] #48750
sum( dtrain$ganancia )

Puedo asignar valores a un dtrain "filtrado". En este caso imputo 0 en los NA

In [ ]:
#Puedo asignar valores a cualquier campo que cumpla con una condici. Ejemplo valores NA
dtrain[is.na(dtrain)] <- 0

# Resumen básico del dtrain
summary( dtrain)

In [ ]:
#Resumen segregado por clase ternaria
by(dtrain, dtrain$clase_ternaria, summary)

## EDA


In [ ]:
hist(dtrain$cliente_edad)

In [ ]:
#Se comportan diferente?
ftable(dtrain[ cliente_edad <=33, clase])

In [ ]:
ftable(dtrain[ cliente_edad  >33, clase])

In [ ]:
#Ganancia
sum( dtrain[ cliente_edad <=33, ganancia] )

In [ ]:
#Lift
(sum( dtrain[ cliente_edad <=33, clase]) / nrow( dtrain[ cliente_edad <=33, ] )) /  (sum( dtrain[, clase]) / nrow( dtrain ))

In [ ]:
#Ganancia
sum( dtrain[ cliente_edad >33, ganancia] )


In [ ]:
#Lift
(sum( dtrain[ cliente_edad >33, clase]) / nrow( dtrain[ cliente_edad >33, ] )) /  (sum( dtrain[, clase]) / nrow( dtrain ))

In [ ]:
#Corte por mcuentas_saldo
hist(dtrain[ ,mcuentas_saldo] )
boxplot(mcuentas_saldo  ~ clase_ternaria, data=dtrain)
boxplot(mcuentas_saldo  ~ clase_ternaria, data=dtrain, outline=FALSE)

In [ ]:
ftable(dtrain[ mcuentas_saldo <= -120000, clase_ternaria])

In [ ]:
ftable(dtrain[ mcuentas_saldo >  -120000, clase_ternaria])

In [ ]:
sum( dtrain[ mcuentas_saldo <= -120000, ganancia] )

In [ ]:
sum( dtrain[ mcuentas_saldo > -120000, ganancia] )

In [ ]:
boxplot(Visa_mconsumototal  ~ clase_ternaria, data=dtrain, outline=FALSE)

In [ ]:
hist(dtrain$cliente_antiguedad)

In [ ]:
hist(dtrain$ccallcenter_transacciones)

In [ ]:
hist(dtrain$chomebanking_transacciones)

In [ ]:
ggplot(dtrain, aes(x=ccallcenter_transacciones, y=chomebanking_transacciones)) + # fill=name allow to automatically dedicate a color for each group
        geom_point()+
        scale_x_log10()+
        scale_y_log10()


In [ ]:
ggplot(dtrain, aes(x=dtrain$Visa_mconsumototal))+ geom_histogram() + scale_x_log10()

In [ ]:
ggplot(dtrain, aes(x=dtrain$Master_mconsumototal))+ geom_histogram() + scale_x_log10()

Determino correlaciones entre las variables y el objetivo. 

In [ ]:
dtrain[ ,  (kcampos_a_borrar) := NULL    ] 
correlaciones <- abs(cor(dtrain[,-..kobjetivo],dtrain[,..kobjetivo]))

In [ ]:
correlaciones[order(-correlaciones[,1]),][0:20]

In [ ]:
library(ggplot2)

for (feature in names(correlaciones[order(-correlaciones[,1]),][c(3,4,6,11)])) {
    
    
    plot <- ggplot(dtrain, aes(x=as.factor(clase), y=get(feature), fill=as.factor(clase))) + # fill=name allow to automatically dedicate a color for each group
      geom_boxplot(fill="slateblue", alpha=0.2)+
      geom_violin(alpha=0.2)+
    ggtitle(feature)+
        scale_y_log10()
    print(plot)
    
}

## Modelo básico
Vamos a definir un modelo muy básico. La gente que no usa las tarjetas de crédito es candidata a darse de baja. Defino una variable como la suma de transacciones en ambas franquicias.

In [ ]:
dtrain$TC_cactividad = dtrain$Visa_cconsumos + dtrain$Master_cconsumos 

# Most basic violin chart
ggplot(dtrain, aes(x=as.factor(clase), y=TC_cactividad, fill=as.factor(clase))) + # fill=name allow to automatically dedicate a color for each group
  geom_boxplot(fill="slateblue", alpha=0.2)+
  geom_violin(alpha = 0.2)+
     ylim(0, 15)

Genero la variable Predicted con 1 si no tiene movimientos. 

In [ ]:
#Inputo missings
dtrain[is.na(dtrain)] <- 0

dtrain[, TC_cactividad := Visa_cconsumos + Master_cconsumos ]
dtrain[ , Predicted  := as.integer(TC_cactividad < 1) ]


Determino la curva ROC y Área bajo la curva

In [ ]:
#Calculo las metricas
pred <- prediction(dtrain$Predicted, dtrain$clase)

#Calculo las metricas
perf <- performance(pred, measure = "tpr", x.measure = "fpr")

#Grafico la ROC
plot(perf, col=rainbow(10))

#Determino AUC
performance( pred,"auc")@y.values


Cuanto dinero vale el modelo para el banco?

In [ ]:
#Calculo la ganancia
sum( dtrain[ dtrain$Predicted==1, ganancia] )

In [ ]:
#TP
sum(dtrain$Predicted*dtrain[,..kobjetivo])

In [ ]:
#Incentivados Totales
sum(dtrain$Predicted)

Corremos el modelo para el dataset del que no conocemos la clase

In [ ]:
#Identifico casos a incentivar en el modelo definido
dapply[, TC_cactividad := Visa_cconsumos + Master_cconsumos ]
dapply[ , Predicted  := as.integer(TC_cactividad < 1) ]

#Inputo missings
dapply[is.na(dapply)] <- 0

#Grabo prediccion en salida
fwrite( dapply[ , list(numero_de_cliente, Predicted) ], #solo los campos para Kaggle
        file= "./exp/EDA/1TRX.csv", 
        sep= "," )

Modelo alternativo con regla mas amplia. Incentivo todos aquellos que tienen menos de tres trx

In [ ]:
dtrain[ , Predicted  := as.integer(TC_cactividad < 3) ]
pred <- prediction(dtrain$Predicted, dtrain$clase)

#Calculo las metricas
perf <- performance(pred, measure = "tpr", x.measure = "fpr")

#Grafico la ROC
plot(perf, col=rainbow(10))

#Determino AUC
performance( pred,"auc")@y.values

#Calculo la ganancia
print(paste("La Ganancia es:", sum( dtrain[ dtrain$Predicted==1, ganancia] )))

In [ ]:
#Identifico casos a incentivar en el modelo definido
dapply[, TC_cactividad := Visa_cconsumos + Master_cconsumos ]
dapply[ , Predicted  := as.integer(TC_cactividad < 3) ]

#Grabo prediccion en salida
fwrite( dapply[ , list(numero_de_cliente, Predicted) ], #solo los campos para Kaggle
        file= "./exp/EDA/3TRX.csv", 
        sep= "," )


Modelo alternativo con regla mas estricta. Incentivo todos aquellos que no tienen trx pero si en el trimestre

In [ ]:
dtrain[ , Predicted  :=  as.integer((mcuentas_saldo < -1000)
                               & (ctrx_quarter < 12)
                              ) ]

pred <- prediction(dtrain$Predicted, dtrain$clase)

perf <- performance(pred, measure = "tpr", x.measure = "fpr")
plot(perf, col=rainbow(12))
performance( pred,"auc")@y.values

#Calculo la ganancia
print(paste("La Ganancia es:", sum( dtrain[ dtrain$Predicted==1, ganancia] )))

In [ ]:
#Identifico casos a incentivar en el modelo definido
dapply[ , Predicted  :=  as.integer((mcuentas_saldo < -1000) 
                               & (ctrx_quarter < 10)
                              ) ]

#Grabo prediccion en salida
fwrite( dapply[ , list(numero_de_cliente, Predicted) ], #solo los campos para Kaggle
        file= "./exp/EDA/Basico_restrictivo.csv", 
        sep= "," )


